In [1]:
# %% [code]
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import time
import os
import traceback # Para imprimir errores detallados


In [ ]:
DIRECCION_DATOS = 'Datos'
DIRECCION_B2C_TRANSACCIONES = os.path.join(DIRECCION_DATOS, 'transacciones_con_features.csv')



# Variables Globales para Datos Pre-calculados 

In [ ]:
user_item_matrix_csr = None
item_similarity_matrix = None
user_to_idx = None
idx_to_user = None
item_to_idx = None
idx_to_item = None
n_users = 0
n_items = 0
data_ready = False # Bandera para indicar si la configuración fue exitosa

# Función para Cargar y Preparar Datos (Adaptada del script original)

In [ ]:

def setup_recommendation_data():
    """Carga datos y calcula matrices UNA VEZ al inicio."""
    global user_item_matrix_csr, item_similarity_matrix
    global user_to_idx, idx_to_user, item_to_idx, idx_to_item
    global n_users, n_items, data_ready

    print("--- Configurando Datos para Recomendación Basada en Ítems ---")
    overall_start_time = time.time()

    # --- 1. Cargar y Preparar Datos ---
    print("Paso 1: Cargando Datos de Transacciones...")
    start_time = time.time()
    try:
        if not os.path.exists(DIRECCION_B2C_TRANSACCIONES):
             # Usar una excepción para detener el proceso si el archivo no está
             raise FileNotFoundError(f"Archivo de transacciones no encontrado: {DIRECCION_B2C_TRANSACCIONES}")

        # Leer solo las columnas necesarias
        transacciones = pd.read_csv(
            DIRECCION_B2C_TRANSACCIONES,
            encoding='utf-8',
            usecols=['id', 'producto'] # Columnas del archivo B2C original
        )
        # Renombrar columna 'id' a 'cliente_id' por claridad
        transacciones.rename(columns={'id': 'cliente_id'}, inplace=True)
        # Eliminar filas con valores faltantes en columnas clave
        transacciones.dropna(subset=['cliente_id', 'producto'], inplace=True)
        # Convertir cliente_id a numérico (entero), manejando errores y eliminando fallos
        transacciones['cliente_id'] = pd.to_numeric(transacciones['cliente_id'], errors='coerce')
        transacciones.dropna(subset=['cliente_id'], inplace=True)
        transacciones['cliente_id'] = transacciones['cliente_id'].astype(int)
         # Limpiar IDs de producto por si acaso (espacios extra)
        transacciones['producto'] = transacciones['producto'].astype(str).str.strip()

        # Eliminar duplicados (si un usuario compró el mismo producto varias veces, solo cuenta como una interacción)
        initial_rows = len(transacciones)
        transacciones = transacciones.drop_duplicates(subset=['cliente_id', 'producto'])
        deduplicated_rows = len(transacciones)
        print(f"Cargadas {initial_rows} interacciones, {deduplicated_rows} únicas usuario-producto.")
        print(f"Paso 1 finalizado en {time.time() - start_time:.2f} segundos.")
    except Exception as e:
        print(f"ERROR CRÍTICO en Paso 1 (Cargar Datos): {e}")
        traceback.print_exc()
        data_ready = False
        return # Detener la configuración

    # --- 2. Crear Mapeos ---
    print("Paso 2: Creando Mapeos de Usuario y Producto...")
    start_time = time.time()
    try:
        # Obtener usuarios y productos únicos y ordenarlos para mapeos consistentes
        unique_clientes = sorted(transacciones['cliente_id'].unique())
        unique_productos = sorted(transacciones['producto'].unique())
        n_users = len(unique_clientes)
        n_items = len(unique_productos)

        if n_users == 0 or n_items == 0:
             print("ERROR CRÍTICO: No se encontraron usuarios o ítems únicos después de procesar los datos.")
             data_ready = False
             return

        # Crear diccionarios para mapear IDs a índices y viceversa
        user_to_idx = {user_id: i for i, user_id in enumerate(unique_clientes)}
        idx_to_user = {i: user_id for user_id, i in user_to_idx.items()}
        item_to_idx = {item_id: i for i, item_id in enumerate(unique_productos)}
        idx_to_item = {i: item_id for item_id, i in item_to_idx.items()}
        print(f"Encontrados {n_users} usuarios únicos y {n_items} ítems únicos.")
        print(f"Paso 2 finalizado en {time.time() - start_time:.2f} segundos.")
    except Exception as e:
        print(f"ERROR CRÍTICO en Paso 2 (Mapeos): {e}")
        traceback.print_exc()
        data_ready = False
        return

    # --- 3. Construir Matriz de Interacción Usuario-Ítem ---
    print("Paso 3: Construyendo Matriz de Interacción Usuario-Ítem...")
    start_time = time.time()
    try:
        # Usar lil_matrix para construcción eficiente (asignación de elementos individuales)
        # Tamaño: n_users x n_items
        user_item_matrix_lil = lil_matrix((n_users, n_items), dtype=np.int8) # int8 para ahorrar memoria (0 o 1)

        # Iterar sobre las transacciones únicas para llenar la matriz
        # Es más eficiente iterar sobre filas de DataFrame para este caso
        for index, row in transacciones.iterrows():
            user_id = row['cliente_id']
            item_id = row['producto']
            # Obtener los índices correspondientes usando los mapeos
            # Verificar que los IDs existan en los mapeos (deberían si vienen de unique(), pero seguridad)
            if user_id in user_to_idx and item_id in item_to_idx:
                user_idx = user_to_idx[user_id]
                item_idx = item_to_idx[item_id]
                user_item_matrix_lil[user_idx, item_idx] = 1 # Marcar interacción con 1

        # Convertir a CSR (Compressed Sparse Row) para cálculos más rápidos como la similitud
        user_item_matrix_csr = user_item_matrix_lil.tocsr()
        del user_item_matrix_lil # Liberar memoria de la matriz LIL
        del transacciones # Liberar memoria del DataFrame de transacciones si no se necesita más

        # Calcular densidad de la matriz (qué porcentaje de celdas no son cero)
        total_possible_interactions = n_users * n_items
        density = user_item_matrix_csr.nnz / total_possible_interactions if total_possible_interactions > 0 else 0
        print(f"Matriz Usuario-Ítem construida ({user_item_matrix_csr.shape}), Elementos no cero (interacciones): {user_item_matrix_csr.nnz}, Densidad: {density:.6f}")
        print(f"Paso 3 finalizado en {time.time() - start_time:.2f} segundos.")
    except Exception as e:
        print(f"ERROR CRÍTICO en Paso 3 (Matriz Usuario-Ítem): {e}")
        traceback.print_exc()
        user_item_matrix_csr = None # Asegurar que sea None en caso de error
        data_ready = False
        return

    # --- 4. Calcular Matriz de Similitud Ítem-Ítem ---
    print("Paso 4: Calculando Matriz de Similitud Ítem-Ítem...")
    start_time = time.time()
    # Solo calcular si la matriz Usuario-Ítem se creó correctamente y tiene datos
    if user_item_matrix_csr is None or user_item_matrix_csr.nnz == 0 or n_items == 0:
        print("Advertencia: Omitiendo cálculo de Similitud Ítem-Ítem (Matriz Usuario-Ítem vacía/inválida o 0 ítems).")
        item_similarity_matrix = None
    else:
        try:
            # Para calcular la similitud entre ítems, necesitamos la matriz en formato Ítem x Usuario.
            # La matriz Usuario-Ítem es Usuarios x Ítems. La transponemos.
            item_user_matrix_csr = user_item_matrix_csr.T.tocsr()

            # Calcular similitud coseno entre ítems (filas de item_user_matrix_csr)
            # Esto nos dará una matriz de n_items x n_items
            # Usar dense_output=True para obtener una matriz densa (numpy array), más fácil de indexar
            temp_item_similarity_matrix = cosine_similarity(item_user_matrix_csr, dense_output=True)

            # Poner la similitud de un ítem consigo mismo a 0, ya que no queremos recomendar el mismo ítem.
            np.fill_diagonal(temp_item_similarity_matrix, 0)

            item_similarity_matrix = temp_item_similarity_matrix # Asignar a la variable global

            print(f"Matriz de similitud Ítem-Ítem calculada ({item_similarity_matrix.shape}).")
            print(f"Paso 4 finalizado en {time.time() - start_time:.2f} segundos.")
        except Exception as e:
            print(f"ERROR CRÍTICO en Paso 4 (Similitud Ítem-Ítem): {e}")
            traceback.print_exc()
            item_similarity_matrix = None # Asegurar que sea None en caso de error

    # --- Verificación Final del Setup ---
    # Comprobar si los componentes esenciales están listos
    if user_item_matrix_csr is not None and item_similarity_matrix is not None and user_to_idx and item_to_idx and n_users > 0 and n_items > 0:
        data_ready = True # Marcar como listo si todo está bien
        print(f"--- Configuración Ítem-Based COMPLETADA en {time.time() - overall_start_time:.2f} segundos. Datos listos: {data_ready} ---")
    else:
        data_ready = False # Marcar como no listo si algo falló
        print(f"--- Configuración Ítem-Based FALLIDA después de {time.time() - overall_start_time:.2f} segundos. Datos listos: {data_ready} ---")

In [5]:
setup_recommendation_data()

# Después de ejecutar, puedes verificar el estado de la bandera global
print(f"\nEstado final de la carga de datos: {data_ready}")

--- Configurando Datos para Recomendación Basada en Ítems ---
Paso 1: Cargando Datos de Transacciones...
Cargadas 2099287 interacciones, 1782324 únicas usuario-producto.
Paso 1 finalizado en 6.08 segundos.
Paso 2: Creando Mapeos de Usuario y Producto...
Encontrados 419221 usuarios únicos y 7276 ítems únicos.
Paso 2 finalizado en 1.00 segundos.
Paso 3: Construyendo Matriz de Interacción Usuario-Ítem...
Matriz Usuario-Ítem construida ((419221, 7276)), Elementos no cero (interacciones): 1782324, Densidad: 0.000584
Paso 3 finalizado en 134.54 segundos.
Paso 4: Calculando Matriz de Similitud Ítem-Ítem...
Matriz de similitud Ítem-Ítem calculada ((7276, 7276)).
Paso 4 finalizado en 1.68 segundos.
--- Configuración Ítem-Based COMPLETADA en 143.30 segundos. Datos listos: True ---

Estado final de la carga de datos: True


# Algoritmo de recomendación

In [ ]:

def get_item_based_recommendations_allow_repurchase(target_cliente_id, N=10, k_similar_items=30):
    """
    Genera recomendaciones de productos basadas en la similitud de ítems con el historial de compras,
    PERMITIENDO que ítems ya comprados sean recomendados nuevamente.

    Args:
        target_cliente_id (int): El ID del cliente para el que se quieren recomendaciones.
        N (int): El número de recomendaciones a devolver.
        k_similar_items (int): El número de ítems más similares a considerar por cada ítem comprado.

    Returns:
        pandas.DataFrame: DataFrame con 'producto' recomendado y 'recommendation_score'.
                          Retorna DataFrame vacío si hay errores, el usuario no existe,
                          no tiene historial, o no se encuentran ítems similares.
    """
    recommendations = pd.DataFrame() # Inicializar DataFrame vacío

    # --- Validación de Entrada y Estado ---
    if not data_ready:
        print("Error: Los datos de recomendación no están listos. Ejecuta la celda de setup.")
        return recommendations
    if target_cliente_id not in user_to_idx:
        print(f"Error: Cliente ID '{target_cliente_id}' no encontrado en el mapeo de usuarios.")
        return recommendations
    if item_similarity_matrix is None:
         print("Error: La matriz de similitud Ítem-Ítem no está disponible para recomendaciones.")
         return recommendations
    if user_item_matrix_csr is None:
         print("Error: La matriz Usuario-Ítem no está disponible.")
         return recommendations


    # Obtener el índice interno del usuario
    target_user_idx = user_to_idx[target_cliente_id]

    # --- Obtener Historial de Compras del Usuario ---
    try:
        # Verificar que el índice del usuario sea válido para la matriz
        if target_user_idx >= user_item_matrix_csr.shape[0]:
             print(f"Error: Índice de usuario {target_user_idx} fuera de los límites de la matriz Usuario-Ítem ({user_item_matrix_csr.shape[0]} filas).")
             return recommendations
        # Obtener los índices de los ítems comprados por el usuario (de la fila correspondiente en la matriz CSR)
        user_purchased_items_indices = user_item_matrix_csr[target_user_idx].indices # Indices de columnas (ítems) donde la fila del usuario no es cero (compró)

        # Si el historial está vacío, no se pueden generar recomendaciones basadas en él
        if len(user_purchased_items_indices) == 0:
            print(f"El usuario {target_cliente_id} (idx {target_user_idx}) no tiene historial de compras en los datos cargados.")
            return recommendations
        print(f"Usuario {target_cliente_id} (idx {target_user_idx}) compró {len(user_purchased_items_indices)} ítems.")

    except Exception as e:
        print(f"Error al recuperar el historial de compras para el usuario {target_cliente_id} (idx {target_user_idx}): {e}")
        traceback.print_exc()
        return recommendations

    # Usar un defaultdict para acumular puntajes para cada ítem candidato
    candidate_items = defaultdict(float) # índice_ítem -> puntaje_acumulado
    processed_purchased_items = 0 # Contador para seguimiento

    try:
        # Iterar sobre cada ítem que el usuario ha comprado
        for purchased_item_idx in user_purchased_items_indices:
            processed_purchased_items += 1
             # Verificar que el índice del ítem comprado sea válido para la matriz de similitud
            if purchased_item_idx < 0 or purchased_item_idx >= item_similarity_matrix.shape[0]:
                 print(f"Advertencia: Índice de ítem comprado {purchased_item_idx} fuera de límites para matriz de similitud ({item_similarity_matrix.shape[0]} filas). Saltando.")
                 continue # Omitir este ítem si su índice no es válido

            # Obtener los puntajes de similitud de este ítem comprado con todos los demás ítems
            # Esto es la fila 'purchased_item_idx' de la matriz de similitud Ítem-Ítem
            item_similarities_vector = item_similarity_matrix[purchased_item_idx]


            similar_item_indices_sorted = np.argsort(item_similarities_vector)[::-1]
            top_k_indices = similar_item_indices_sorted[:k_similar_items]


            # Iterar sobre los ítems más similares encontrados para este ítem comprado
            for similar_item_idx in top_k_indices:
                # Obtener el puntaje de similitud entre el ítem comprado y el ítem similar
                similarity_score = item_similarities_vector[similar_item_idx]

                # La similitud 0 ya se ha puesto en la diagonal, pero si hay otros ítems con similitud 0 o negativa, saltar.
                if similarity_score <= 0:
                    continue

                # Acumular puntaje para el ítem similar. Sumamos la similitud.
                candidate_items[similar_item_idx] += similarity_score

        # Si después de iterar sobre todos los ítems comprados, no hay candidatos acumulados...
        if not candidate_items:
             print(f"Advertencia: No se encontraron ítems candidatos con similitud positiva para el usuario {target_cliente_id} (basado en {processed_purchased_items} ítems comprados).")
             return recommendations

        print(f"Procesados {processed_purchased_items} ítems comprados. Encontrados {len(candidate_items)} ítems candidatos.")

        # --- Rankear Candidatos ---
        # Convertir el diccionario de candidatos (índice -> puntaje acumulado) a una lista de diccionarios
        ranked_candidates = []
        for item_idx, score in candidate_items.items():
             item_id = idx_to_item.get(item_idx) # Convertir índice interno de nuevo a ID de producto
             if item_id: # Asegurarse de que el mapeo inverso funcione
                 # Asegurar que el puntaje sea un float estándar para evitar problemas de serialización JSON/otros
                 ranked_candidates.append({'producto': item_id, 'recommendation_score': float(score)})
             else:
                 print(f"Advertencia: No se encontró ID de producto para el índice {item_idx}. Saltando.")


        # Si no se pudo mapear ningún índice o no había candidatos válidos, devolver vacío
        if not ranked_candidates:
             print("Advertencia: No se pudieron rankear candidatos válidos.")
             return recommendations


        # Crear DataFrame, ordenar por puntaje descendente y tomar los N mejores
        recommendations = pd.DataFrame(ranked_candidates)
        recommendations = recommendations.sort_values('recommendation_score', ascending=False).head(N)

    except Exception as e:
         print(f"Error generando y rankeando candidatos para el usuario {target_cliente_id}: {e}")
         traceback.print_exc()
         return pd.DataFrame() # Devolver DataFrame vacío en caso de error

    return recommendations

# Ejemplo de uso

In [9]:

# Elige un ID de cliente que sepas que existe en tus datos B2C.
# El ID 6 aparece en tu ejemplo de respuesta.
example_cliente_id = 6


# Define cuántas recomendaciones quieres
num_recommendations = 10
num_similar_items_to_consider = 30 # Puedes ajustar este valor

print(f"Intentando obtener recomendaciones para Cliente ID: {example_cliente_id}")

# Llama a la función de recomendación
start_time_rec = time.time()
recommendations = get_item_based_recommendations_allow_repurchase(
    target_cliente_id=example_cliente_id,
    N=num_recommendations,
    k_similar_items=num_similar_items_to_consider
)
end_time_rec = time.time()


# --- Mostrar Resultados ---
if recommendations is None:
    print("La función de recomendación devolvió None (hubo un error crítico).")
elif recommendations.empty:
    print(f"No se encontraron recomendaciones para el Cliente ID {example_cliente_id}.")
else:
    print("\n--- Recomendaciones Encontradas ---")
    print(recommendations)
    print(f"\nRecomendaciones generadas en {end_time_rec - start_time_rec:.4f} segundos.")



Intentando obtener recomendaciones para Cliente ID: 6
Usuario 6 (idx 5) compró 16 ítems.
Procesados 16 ítems comprados. Encontrados 252 ítems candidatos.

--- Recomendaciones Encontradas ---
        producto  recommendation_score
30   producto_49              0.786332
35  producto_119              0.653563
38   producto_19              0.619442
34    producto_3              0.482461
25   producto_28              0.478430
31  producto_248              0.350241
65  producto_176              0.344945
13   producto_27              0.334039
68   producto_40              0.316651
79   producto_66              0.285180

Recomendaciones generadas en 0.0103 segundos.
